In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_k


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="destination", description="여행 후기나 계획 텍스트를 입력 받아 여행지 출력"),
    ResponseSchema(name="duration", description="여행 후기나 계획 텍스트를 입력 받아 기간 출력"),
    ResponseSchema(name="budget", description="여행 후기나 계획 텍스트를 입력 받아 예산 출력"),
    ResponseSchema(name="rating", description="여행 후기나 계획 텍스트를 입력 받아 추천도(1~5점) 출력"),
    ResponseSchema(name="activities", description="여행 후기나 계획 텍스트를 입력 받아 주요 활동을 리스트로 출력")
]

In [5]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행 후기나 계획 텍스트를 입력 받아 여행지 출력
	"duration": string  // 여행 후기나 계획 텍스트를 입력 받아 기간 출력
	"budget": string  // 여행 후기나 계획 텍스트를 입력 받아 예산 출력
	"rating": string  // 여행 후기나 계획 텍스트를 입력 받아 추천도(1~5점) 출력
	"activities": string  // 여행 후기나 계획 텍스트를 입력 받아 주요 활동을 리스트로 출력
}
```


In [6]:
# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요.
 정말 만족스러운 여행이었습니다.
 5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

In [8]:
# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
print(type(output))
pprint(output)

===== 분석 결과 =====
<class 'dict'>
{'activities': ['해운대 바다 구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4점'}
